In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
data = pd.read_csv('arbitrage_data.tsv', names=['spread', 'star_buy_price', 'start_sell_price', 'end_buy_price', 'end_sell_price', 'date_start', 'date_end',
                                                'ticker', 'exchange_buy', 'exchnage_sell'])
data['date_start'] = data['date_start'].apply(lambda x: datetime.fromtimestamp(x))
data['date_end'] = data['date_end'].apply(lambda x: datetime.fromtimestamp(x))
data['total_time'] = [date_end - date_start for date_start, date_end in zip(data['date_start'].values, data['date_end'].values)]
data['total_time'] = data['total_time'].apply(lambda x: x.total_seconds())
data

,spread,star_buy_price,start_sell_price,end_buy_price,end_sell_price,date_start,date_end,ticker,exchange_buy,exchnage_sell,total_time
0,42.334,0.029750,0.051000,0.022680,0.022760,2022-11-01 00:00:00,2022-11-23 04:01:00,MDT/USDT,binance,mexc,1915260.0
1,1.500,0.022660,0.023000,0.022620,0.022750,2022-11-23 04:07:00,2022-11-23 04:07:00,MDT/USDT,binance,mexc,0.0
2,2.080,0.022630,0.023100,0.022630,0.022700,2022-11-23 04:13:00,2022-11-23 04:13:00,MDT/USDT,binance,mexc,0.0
3,1.190,0.022630,0.022900,0.022630,0.022800,2022-11-23 04:16:00,2022-11-23 04:16:00,MDT/USDT,binance,mexc,0.0
4,1.433,0.022810,0.023090,0.022940,0.023090,2022-11-23 07:04:00,2022-11-23 07:11:00,MDT/USDT,binance,mexc,420.0
...,...,...,...,...,...,...,...,...,...,...,...
976516,1.230,0.000021,0.000021,0.000021,0.000021,2022-11-30 20:33:00,2022-11-30 20:34:00,XRDOGE/USDT,bitrue,whitebit,60.0
976517,1.065,0.000021,0.000021,0.000021,0.000021,2022-11-30 20:37:00,2022-11-30 20:38:00,XRDOGE/USDT,bitrue,whitebit,60.0
976518,1.090,0.000021,0.000021,0.000021,0.000021,2022-11-30 20:51:00,2022-11-30 20:51:00,XRDOGE/USDT,bitrue,whitebit,0.0
976519,1.230,0.000021,0.000021,0.000021,0.000021,2022-11-30 21:04:00,2022-11-30 21:07:00,XRDOGE/USDT,bitrue,whitebit,180.0


In [3]:
print(len(data.loc[(data['total_time'] > 180) & (data['total_time'] < 900)]), len(data.loc[(data['total_time'] >= 900)]))
# Без учета дополнительных проверок на связки, ~59% связок нерабочие.
# Однако, если проверять их в реальном времени можно получить больше информации о маркете => более чательно отобрать ситуации
# Конечно их будет намного меньше, но процент нерабочих связок будет около 5%

220557 158446


In [4]:
data = data.loc[(data['total_time'] >= 900) & (data['spread'] <= 50)]
data.reset_index(inplace=True)
data.drop('index', axis=1, inplace=True)
data.head(2)

C:\Users\Miships\AppData\Local\Temp\ipykernel_17176\978192652.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('index', axis=1, inplace=True)


,spread,star_buy_price,start_sell_price,end_buy_price,end_sell_price,date_start,date_end,ticker,exchange_buy,exchnage_sell,total_time
0,42.334,0.02975,0.05100,0.02268,0.02276,2022-11-01 00:00:00,2022-11-23 04:01:00,MDT/USDT,binance,mexc,1915260.0
1,1.657,0.02290,0.02331,0.02292,0.02175,2022-11-23 07:33:00,2022-11-23 07:59:00,MDT/USDT,binance,mexc,1560.0


In [12]:
facts = []
for _ in range(10000):
    index = np.random.choice(data.index, size=1000, replace=False)
    s = sum([data['spread'][ind] for ind in index])
    facts.append(s)
facts.sort()
print(f'from {int(facts[249])} to {int(facts[-250])}')

from 2549 to 2958
